In [2]:
from pathlib import PurePath
from pydub import AudioSegment
import wave
import contextlib
import os
import random

In [3]:
path_clean_flac = 'clean_testset_flac\\'
path_clean_wav = 'clean_testset_wav\\'
path_noises = 'noises\\'
path_noisy_wav = 'noisy_testset_wav\\'

In [4]:
def fluc_to_wav(flac):
    file = AudioSegment.from_file(flac, flac.split('.')[1])
    flac = flac.replace(path_clean_flac, path_clean_wav)
    file.export(flac.replace(".flac", ".wav"), format="wav")
    return flac.replace(".flac", ".wav")

In [5]:
def cut_mix_wavs(wav1, wav2, filename):
    with contextlib.closing(wave.open(wav2,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration1 = frames / float(rate)
    with contextlib.closing(wave.open(wav1,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration2 = frames / float(rate)
    t1 = int(random.randrange(0, int(duration2 - duration1)-1, 1)) * 1000 #Works in milliseconds
    t2 = (t1 + duration2) * 1000
    cutedAudio = AudioSegment.from_wav(wav1)
    cutedAudio = cutedAudio[t1:t2]
    mix_wavs(cutedAudio, wav2, filename)


In [6]:
def mix_wavs(segmentwav1, wav2, filename):
    sound1 = AudioSegment.from_file(wav2)
    sound2 = segmentwav1

    combined = sound1.overlay(sound2)

    combined.export(path_noisy_wav+filename, format='wav')
    return

In [7]:

def main():
    for root, dirs, files in os.walk(path_clean_flac):
        for filename in files:
            fluc_to_wav(path_clean_flac + filename)
    noises_arr = []    
    for _, _, noises in os.walk(path_noises):
            for noise in noises:
                    noises_arr.append(noise)
    for root, dirs, files in os.walk(path_clean_wav):
        for filename in files:
            cut_mix_wavs(path_noises+noises_arr[int(random.randrange(0, len(noises_arr)-1, 1))], path_clean_wav+filename, filename)
                    

In [8]:
main()